In [1]:
import pandas as pd

In [2]:
pickup_geohash = 'dr5ru2'
dropoff_geohash = 'dr5rvj'
q = """
SELECT
  pickup_street1, pickup_street2, dropoff_street1, dropoff_street2,
  trips_medallion, trips_pickup_datetime, trips_dropoff_datetime,
  ROUND(trips_avg_mph, 4) AS avg_mpg,
  ROUND(trips_trip_duration_hours, 4) AS num_hours
FROM
  [taxi_strava.joined_geohash_geonames]
WHERE
  trips_geohashed_pickup = '{0}'
  AND trips_geohashed_dropoff = '{1}'
""".format(pickup_geohash, dropoff_geohash)

In [3]:
df = pd.read_gbq(q, project_id='bigquery-reddit-1003')

DistributionNotFound: The 'google-api-python-client' distribution was not found and is required by the application